In [3]:
import sys

print(sys.version)
print(sys.executable)

3.8.5 (default, Dec 15 2020, 06:10:15) 
[GCC 5.4.0 20160609]
/home/sage/sage/local/bin/python3


From https://ericpony.github.io/z3py-tutorial/guide-examples.htm

In [4]:
from z3 import *

# Solving Equations

To solve:

* $x > 2$
* $y < 2$
* $x + 2 y = 7$

In [8]:
x = Int('x')
y = Int('y')

c1 = x > 2
c2 = y < 2
c3 = x + 2 * y == 7

solve(c1, c2, c3)

[y = 0, x = 7]


# Simplifying Expressions

In [9]:
x = Int('x')
y = Int('y')

In [10]:
expr1 = x + y + 2 * x + 3
expr2 = x < y + x + 2
expr3 = And(x + 1 >= 3, x ** 2 + x **2 + y ** 2 + 2 >=5)

In [33]:
expr1

x + y + 2*x + 3

In [34]:
simplify(expr1)

3 + 3*x + y

In [35]:
expr2

x < y + x + 2

In [36]:
simplify(expr2)

Not(y <= -2)

In [37]:
expr3

And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)

In [38]:
simplify(expr3)

And(x >= 2, 2*x**2 + y**2 >= 3)

# Picking Expressions Apart

In [41]:
x = Int('x')
y = Int('y')

In [42]:
n = x + y >= 3
n

x + y >= 3

In [43]:
n.num_args()

2

In [44]:
n.children()

[x + y, 3]

In [45]:
n.decl() # the operator

>=

In [46]:
n.arg(0)

x + y

In [47]:
n.arg(1)

3

In [48]:
n.arg(0).decl()

+

In [49]:
n.arg(0).decl().name()

'+'

# Solving Nonlinear Polynomial Constraints

In [50]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 > 3, x**3 + y < 5)

[y = 2, x = 1/8]


# Configuring The Environment

In [68]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

set_param(precision=30)

solve(x**2 + y**2 == 3, x**3 == 2)

[y = -1.188528059421316533710369365015?,
 x = 1.259921049894873164767210607278?]
[y = -1.188528059421316533710369365015?,
 x = 1.259921049894873164767210607278?]


# Boolean Logic

Prove [demorgan's law](https://en.wikipedia.org/wiki/De_Morgan%27s_laws):

In [73]:
p = Bool('p')
q = Bool('q')

In [75]:
# note that in Z3, <=> is written as ==
dmlaw1 = Not(Or(p, q)) == And(Not(p), Not(q))
dmlaw2 = Not(And(p, q)) == Or(Not(p), Not(q))

In [77]:
solve(Not(dmlaw1))

no solution


In [79]:
solve(Not(dmlaw2))

no solution


`Not dmlaw` has no solution means `dmlaw` is true!

Hint:

> A formula/constraint F is valid if F always evaluates to true for any assignment of appropriate values to its uninterpreted symbols. A formula/constraint F is satisfiable if there is some assignment of appropriate values to its uninterpreted symbols under which F evaluates to true. Validity is about finding a proof of a statement; satisfiability is about finding a solution to a set of constraints. Consider a formula F containing a and b. We can ask whether F is valid, that is whether it is always true for any combination of values for a and b. If F is always true, then Not(F) is always false, and then Not(F) will not have any satisfying assignment (i.e., solution); that is, Not(F) is unsatisfiable. That is, F is valid precisely when Not(F) is not satisfiable (is unsatisfiable). Alternately, F is satisfiable if and only if Not(F) is not valid (is invalid). The following example proves the deMorgan's law.

# The Solver API

## Basics

In [86]:
x = Int('x')
y = Int('y')

In [87]:
s = Solver()
s

[]

In [88]:
s.add(x > 10, y == x + 2)
s

[x > 10, y == x + 2]

In [89]:
# check if all constraints can be satisfied
s.check()

sat

In [90]:
# what happens if we add y < 11 ?
s.push()
s.add(y < 11)
s

[x > 10, y == x + 2, y < 11]

In [91]:
s.check()

unsat

In [92]:
# remove the y < 11 constraint
s.pop()
s

[x > 10, y == x + 2]

In [93]:
s.check()

sat

## Traversing Assertions 

In [96]:
x = Real('x')
y = Real('y')
s = Solver()
s.add(x > 1, y > 1, Or(x + y > 3, x - y < 2))

In [97]:
for c in s.assertions():
    print(c)

x > 1
y > 1
Or(x + y > 3, x - y < 2)


## Statistics Of Last `check()`

In [98]:
s.check()

sat

In [99]:
s.statistics()

(:arith-lower         1
 :arith-make-feasible 3
 :arith-max-columns   8
 :arith-max-rows      2
 :arith-upper         3
 :decisions           2
 :final-checks        1
 :max-memory          9.26
 :memory              7.07
 :mk-bool-var         6
 :num-allocs          519904318
 :num-checks          1
 :rlimit-count        23673
 :time                0.02)

In [100]:
for stat,val in s.statistics():
    print(f"{stat}:\t{val}")

mk bool var:	1
decisions:	2
final checks:	1
num checks:	1
mk bool var:	5
arith-lower:	1
arith-upper:	3
arith-make-feasible:	3
arith-max-columns:	8
arith-max-rows:	2
num allocs:	519904318
rlimit count:	23673
max memory:	9.26
memory:	7.07
time:	0.02


## Inspecting A Model

In [101]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
s.check()

sat

In [103]:
m = s.model()
m

[z = 0, y = 2, x = 3/2]

Getting the values of each variable:

In [104]:
for v in [x, y, z]:
    print(f"The value of {v} is {m[v]}")

The value of x is 3/2
The value of y is 2
The value of z is 0


Alternatively:

In [109]:
for decl in m.decls():
    print(f"{decl.name()} = {m[decl]}")

z = 0
y = 2
x = 3/2


In [106]:
m.decls()

[z, y, x]

# Unknown Solution

Nonlinear and not polynomial:

In [94]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
s.check()

unknown

In [95]:
s.model()

[]

# BitVectors

In [110]:
x = BitVec('x', 16)
y = BitVec('y', 16)

In [111]:
x + 2

x + 2

In [114]:
# internal representation
(x + 2).sexpr()

'(bvadd x #x0002)'

In [119]:
# -1 is equal to 65535 for 16-bit integers
simplify(x + y - 1)

65535 + x + y

In [121]:
# bitvector constants
a = BitVecVal(-1, 16)
b = BitVecVal(65535, 16)

print(a == b)
print(simplify(a == b))

65535 == 65535
True


In [123]:
a = BitVecVal(-1, 32)
b = BitVecVal(65535, 32)

# -1 is not equal to 65535 for 32-bit integers 
print(a == b)
print(simplify(a == b))

4294967295 == 65535
False


Arithmetic:

In [124]:
# Create to bit-vectors of size 32
x, y = BitVecs('x y', 32)

solve(x + y == 2, x > 0, y > 0)

# Bit-wise operators
# & bit-wise and
# | bit-wise or
# ~ bit-wise not
solve(x & y == ~y)

solve(x < 0)

[y = 1, x = 1]
[y = 4294967295, x = 0]
[x = 4294967295]


In [125]:
# using unsigned version of < 
solve(ULT(x, 0))

no solution


In [126]:
# Create to bit-vectors of size 32
x, y = BitVecs('x y', 32)

solve(x >> 2 == 3)

[x = 12]


In [127]:
solve(x << 2 == 3)

no solution


In [128]:
solve(x << 2 == 24)

[x = 6]


# Functions

In [132]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())

Can there be an `f` such that applying it twice returns `x` and once returns `y`?

In [133]:
solve(f(f(x)) == x, f(x) == y, x != y)

[x = 0, y = 1, f = [1 -> 0, else -> 1]]


Evaluating solution:

In [134]:
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)

In [135]:
s.check()

sat

In [136]:
m = s.model()
m

[x = 0, y = 1, f = [1 -> 0, else -> 1]]

In [144]:
print(f"The value of x is {m.evaluate(x)}")
print(f"f(x) is {m.evaluate(f(x))}")
print(f"f(f(x)) is {m.evaluate(f(f(x)))}")

print("\n")

print(f"The value of y is {m.evaluate(y)}")
print(f"f(y) is {m.evaluate(f(y))}")
print(f"f(f(y)) is {m.evaluate(f(f(y)))}")

The value of x is 0
f(x) is 1
f(f(x)) is 0


The value of y is 1
f(y) is 0
f(f(y)) is 1


In [146]:
print(f"f(10) is {m.evaluate(f(10))}")
print(f"f(f(10)) is {m.evaluate(f(f(10)))}")

f(10) is 1
f(f(10)) is 0


# Vector Datatype

In [148]:
X = IntVector('x', 5)
Y = RealVector('y', 5)
P = BoolVector('p', 5)
print(X)
print(Y)
print(P)
print([ y**2 for y in Y ])
print(Sum([ y**2 for y in Y ]))

[x__0, x__1, x__2, x__3, x__4]
[y__0, y__1, y__2, y__3, y__4]
[p__0, p__1, p__2, p__3, p__4]
[y__0**2, y__1**2, y__2**2, y__3**2, y__4**2]
y__0**2 + y__1**2 + y__2**2 + y__3**2 + y__4**2


# Proving A Claim

In [152]:
x      = BitVec('x', 32)
y      = BitVec('y', 32)

# Claim: (x ^ y) < 0 iff x and y have opposite signs
trick  = (x ^ y) < 0

# Naive way to check if x and y have opposite signs
opposite = Or(And(x < 0, y >= 0),
              And(x >= 0, y < 0))

prove(trick == opposite)

proved


# Further Reading

* https://github.com/Z3Prover/z3/tree/master/examples/python/tutorial/jupyter